In [24]:
import pandas as pd
import numpy as np
import copy

In [25]:
def minmax_norm(df,lable):
    list_ = df.loc[:,lable].values
    max_ = max(list_)
    min_ = min(list_)
    if max_-min_ == 0:
        return np.ones(len(list_))
    else:
        return (list_ - min_)/(max_-min_)

In [ ]:
data_set = pd.read_excel('./data.xlsx')
data_set.drop(labels=['survey_time','variable'],axis=1,inplace=True)
data_set.info()

In [27]:
#data1 = data_set.fillna(value={'career2':-1})
#data1 = data1.loc[data1.career2 != -1]
#data1 = data1.loc[data1.career1 == 12]
#career_set = list(set(data1['career2'].values))
#1-学生；2-室内工作人员（教师、酒店服务员、IT程序员、行政人员等）；3-室外个体经商者（街边商贩、报亭人员等）；
#4-快递员；5-保安员；6-交警/交通协管；7-加油站、停车场工作人员；8-户外工人（建筑工人、装修工人等）；9-厨师；
#10-司机（出租车、公交车、运输车辆等）；11-外卖送餐员；12-其他

#career_set
gather_dic ={12:['农','务农','农民'],13:['室内个体经营者','个体经营','个体','淘宝商','网店店主'],
              14:['收藏','销售'],
              15:['离退休','退休','退休人员','离退人员', '主妇','无业','家庭主妇'],
              2:['美容行业','金融工作人员','收藏','营业员','生产车间','医生','室内生产'],
              4:['三轮车拉货']}


career_list = data_set.loc[:,['career1','career2']].values
for i in range(len(career_list)):
    if career_list[i][0] != 12:
        pass
    else:
        for k,v in gather_dic.items():
            for career_one in v:
                if career_list[i][1] == career_one:
                    career_list[i][0] = int(k)

print(set(career_list[:,0]))

{2.0, 4.0, 5.0, 8.0, 13.0}


In [28]:
#data2 = data_set.fillna(value={'residence_type2':-1})
#data2 = data2.loc[data2.residence_type2 != -1]
#data2 = data2.loc[data2.residence_type1 == 3.0]
#career_set = list(set(data1['career2'].values))
#residence_list = data_set.loc[:,['residence_type1','residence_type2']].values
#other = []
#for i in range(len(residence_list)):
#    if residence_list[i][0] == 3.0:
#       other.append( residence_list[i][1])
#len(other)331
#数量不足1%，放弃对之的处理
data3 = data_set.fillna(-1)

education_list = data3.loc[:,'education'].values
education_dic = {1:'初中',2:'大学及以上（含大专）',3:'小学及以下',4: '高中/职高（含中专）'}
for i in range(len(education_list)):
    for k,v in education_dic.items():
        if v == education_list[i]:
            education_list[i] = k
            
work_state_list = data3.loc[:,'work_state'].values
temp = list(set(work_state_list))
#删除所有数字
work_state_dic = {1:'待业或无业（全职主妇、家庭主妇、待就业人员）', 2:'离退休', 3:'在职（含自由职业人员、学生）'}
for i in range(len(work_state_list)):
    for k,v in work_state_dic.items():
        if work_state_list[i] == v:
            work_state_list[i] = k
    if int(work_state_list[i]) > 3:
        work_state_list[i] = -1

locationhb_list = data_set.loc[:,'locationhb'].values
locationhb_dic = {1:'indoor',2: 'outdoor', 3:'transport'}
for i in range(len(locationhb_list)):
    for k,v in locationhb_dic.items():
        if locationhb_list[i] == v:
            locationhb_list[i] = k
            
act_list = data_set.loc[:,'act'].values

for i in range(len(act_list)):
    if act_list[i] != '1' and act_list[i] != '2' and act_list[i] != '3' and act_list[i] != 1 and act_list[i] != 2 and act_list[i] != 3:
        act_list[i] = -1
set(act_list)

{-1.0, 1.0}

In [ ]:
data_res = copy.deepcopy(data_set)
data_res.drop(['career1','career2','residence_type2'],axis=1,inplace=True)
data_res['career'] = career_list[:,0]
data_res['education'] = education_list
data_res['work_state'] = work_state_list
data_res['locationhb'] = locationhb_list
data_res['act']=act_list


data_res['height'].where(data_res['height'] < 120 ,-1)
data_res['height'].where( data_res['height'] > 210,-1)

data_res['weight'].where(data_res['weight'] < 30 ,-1)
data_res['weight'].where(data_res['weight'] > 100 ,-1)

data_res[:3]

In [ ]:
data_res['act'] = data_res['act'].astype('int')
data_res['work_state'] = data_res['work_state'].astype('int')
data_res['education'] = data_res['education'].astype('int')
data_res['locationhb'] = data_res['locationhb'].astype('int')
data_res['career'] = data_res['career'].astype('int')


data_res.info()

In [ ]:
#先drop掉residence_floor ,对其他nan赋-1
residence_floor_list = data_res.loc[:,'residence_floor'].values
data_res.drop(labels=['residence_floor','location'],axis=1,inplace=True)
data_res.fillna(-1,inplace=True)

In [ ]:
#删掉所有含有-1的行
#df2[["Weight","Height","BootSize","SuitSize"]].astype(str).replace('0',np.nan)
head = list(data_res.columns)
data_res = data_res[head].replace(['-1',-1,-1.0],np.nan)
data_res.dropna(subset=head, how='any', inplace=True)
data_res.reset_index(inplace=True)
len(data_res)

In [ ]:
#进行归一化处理'residence_floor'
norm_list = ['height','weight', 'work_days', 'rest_days', 
             'conditioner_time', 'kaichuang_time', 'air_cleaner_time', 'age']
for norm_name in norm_list:
    data_res[norm_name] = minmax_norm(data_res,norm_name)

In [ ]:
#进行one-hot处理
one_hot_list = ['gender','education','work_state','career','residence_type1','act']
for one_hot_name in one_hot_list:
    temp_pd = pd.get_dummies(data_res.loc[:,one_hot_name])
    head_temp = list(temp_pd.columns)
    head_temp = [one_hot_name + str(s) for s in head_temp]
    temp_pd.columns = head_temp
    
    #合并和删除
    data_res = pd.concat([data_res,temp_pd],axis=1)
    data_res.drop(labels=one_hot_name,axis=1,inplace=True)

In [ ]:
#进行label处理
#合并smoking_number locationhb
labels_set1 = list(set(data_res.loc[:,'smoking_number'])) 
labels_set2 = list(set(data_res.loc[:,'locationhb'])) 

labels = [str(x) + '_'+ str(y) for x in labels_set1 for y in labels_set2]
data_res['label'] = np.zeros(len(data_res))
for i in range(len(data_res)):
    for j in range(len(labels_set1)):
        if data_res.loc[i,'smoking_number'] == labels_set1[j]:
            for k in range(len(labels_set2)):
                if data_res.loc[i,'locationhb'] == labels_set2[k]:
                    data_res.loc[i,'label'] = len(labels_set2)*j + k + 1
data_res.drop(labels=['smoking_number','locationhb'],axis=1,inplace=True)
            

In [ ]:
#替换tandf
head = list(data_res.columns)
data_res[head] = data_res[head].replace({True: 1, False: 0})
data_res[:10]

In [ ]:
data_res.to_csv('data_res.csv')